<a href="https://colab.research.google.com/github/satyajitghana/TSAI-DeepVision-EVA4.0/blob/master/04_ArchitectureBasics/ArchitectureBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# call upon the python and pytorch gods
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

In [0]:
"""Net: Our Base Model for MNIST Classification
Params
    None
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input - 28x28x1; output - 28x28x32; RF - 1x1
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    """forward: performs a forward pass when model(x) is called
    Params
        x: the input data
    Returns
        y: the output of the model
    """
    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=0)

In [27]:
# check if cuda is available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# model the model to the device
model = Net().to(device)
# print the model summary when given input_size
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

In [0]:
# to get consistent results
torch.manual_seed(1)

# batch size
batch_size = 64

# note about pin_memory
# If you load your samples in the Dataset on CPU and would like to push it
# during training to the GPU, you can speed up the host to device transfer by
# enabling pin_memory. This lets your DataLoader allocate the samples in
# page-locked memory, which speeds-up the transfer.

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# train data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
# this automatically selects tqdm for colab_notebook
from tqdm.auto import tqdm, trange

"""train : trains a given model
Params
    model: the model to train
    device: the device to train on
    train_loader: the train data
    optimizer: the optimizer to use
    epoch: epochs to train for

Returns
    None
"""
def train(model, device, train_loader, optimizer, epoch) -> None:
    # set the model in training mode
    model.train()

    pbar = tqdm(train_loader, ncols="80%")

    # run for all the data in train_loader
    for batch_idx, (data, target) in enumerate(pbar):
        # fetch the data and the target value
        data, target = data.to(device), target.to(device)
        # zero out the gradients
        optimizer.zero_grad()
        # forward pass the model with the data
        output = model(data)
        # compute the loss occured
        loss = F.nll_loss(output, target)
        # compute the backward gradients
        loss.backward()
        # update the optimizer params
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item():.10f} batch_id={batch_idx:05d}')

"""test: tests a given model
Params
    model: the model to test
    device: the device used for testing, cpu/gpu
    test_loader: the test data

Returns
    None
"""
def test(model, device, test_loader) -> None:
    # set the model in evaluation mode
    model.eval()
    # set the loss to zero
    test_loss = 0
    correct = 0

    # turn off gradients during the testing
    with torch.no_grad():
        for data, target in test_loader:
            # move the data to device
            data, target = data.to(device), target.to(device)
            # forward pass the model and get the output
            output = model(data)
            # accumulate the loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # get the index of max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            # increment the correct prediction count if pred is correct
            correct += pred.eq(target.view_as(pred)).sum().item()

    # test loss divided by the total number of items in test dataset
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [34]:
# move the model to the specified device
model = Net().to(device)

# use Stochastic Gradient Descent as the optimizer
# learning_rate=0.01 and momentum=0.9 is standard values for SGD
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# set the number of epochs to train for
N_EPOCH = 2

for epoch in range(1, N_EPOCH+1):
    print('\nEpoch {} : '.format(epoch))
    # train the model
    train(model, device, train_loader, optimizer, epoch)
    # test the model
    test(model, device, test_loader)


Epoch 1 : 



Test set: Average loss: 2.0216, Accuracy: 9800/10000 (98%)


Epoch 2 : 



Test set: Average loss: 1.9822, Accuracy: 9886/10000 (99%)

